In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("<style>div.output_scroll { height: auto; }</style>"))
# TODO need to ctrl-shift-I after that?? wtf?

In [ ]:
import os
from pathlib import Path
pp = str(Path(os.getcwd()).absolute().parent / 'src')
import sys
sys.path.insert(0, pp)

In [ ]:
import my.emfit as emfit
emfit_df = emfit.dataframe()

In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
output_notebook()
from bokeh.models import LinearAxis, Range1d
from bokeh.models import HoverTool
from bokeh.models import ColumnDataSource as CDS
import pandas as pd

In [ ]:
from dashboard.data import weight_dataframe as DF
wdf = DF()

In [ ]:
%autoreload 2
# TODO hmm, maybe this 2 is useful to reload submodules? check later
# yeah, without it seems to throw some crappy errors?

In [ ]:
from dashboard.weight import plot_weight
show(plot_weight(wdf))

In [ ]:
# see https://docs.bokeh.org/en/latest/docs/user_guide/tools.html#basic-tooltips
# TODO date is displayed as unix time
# todo onclick example https://stackoverflow.com/questions/53825387/python-bokeh-update-scatter-plot-colors-on-callback
# TODO determine delta hr and plot 'dependence' on workdays?
# TODO or plot 'derivative'?
# todo disable individual points by default?

In [ ]:
from dashboard.data import sleep_dataframe
df = sleep_dataframe()

In [ ]:
from dashboard.sleep import *
show(plot_all_sleep(df))

In [ ]:
import my.hypothesis as H
hls = H.highlights()

In [ ]:
from dashboard.data import endomondo_dataframe
df = endomondo_dataframe()

In [ ]:
from dashboard.running import plot_running
show(plot_running(df))

In [ ]:
df = pd.DataFrame(h._asdict() for h in hls)
# print(df)


df['count'] = 1 # meh

df['date'] = df['created'].dt.date

ddf = df.groupby(['date'], as_index=False).sum()


p = date_figure()
add_daysoff(p, dates=ddf['date'])
p.scatter(x='created', y='count'  , source=CDS(data=df ), color='blue', legend_label='Hypothesis')
p.vbar   (x='date'   , top='count', source=CDS(data=ddf))


p.legend.click_policy = 'hide'

show(p)

# TODO might be nice to have a slider through a week at a time or something?
# TODO maybe could display other sources on the same plots?

In [ ]:
import my.bluemaestro as bluemaestro
orig_df = bluemaestro.dataframe()

In [ ]:
import datetime
from bokeh.models import ColumnDataSource as CDS

df = orig_df
df = df.sort_index() # FIXME should be fixed in hpi? but also make it defensive
df = df[-40000:]

# TODO 'index must be monotonic??'

print(df[-5:])
m1d  = df[['temp']].rolling('1D' ).mean()

p = figure(x_axis_type='datetime', plot_width=2000)
p.scatter(x='dt', y='temp', source=CDS(data=df ), color='blue', legend_label='Temperature'         )
p.line   (x='dt', y='temp', source=CDS(data=m1d), color='red' , legend_label='Temperature (1d avg)')
# TODO plot frequency of data?? could be even automatic (akin to hpi doctor/stat)

show(p)
# TODO highlight different days?

# TODO hpi: I feel like points duplicate each other without being filtered...
# and probably across different export files?? a bit crap

day_count = df.groupby(df.index.date).count()
print(day_count[:10])

p = figure(x_axis_type='datetime', plot_width=2000)
p.vbar(x='index', top='temp', source=CDS(day_count), width=datetime.timedelta(days=1) * 0.95)
show(p)


In [ ]:
# TODO for temp, maybe heatplot would make more sense? E.g. x axis days y axis time of day. although discontinuity at 0:00 is gonna be annoying.
# could group by weeks?

In [ ]:
# TODO hmm, maybe update regression lines dynamically? Wonder if there is as callback in bokeh
# https://stackoverflow.com/q/53287715/706389 here some sample code
import holoviews as hv
from holoviews import opts
hv.extension('bokeh')

In [ ]:
# regression tap? some callbacky thing
# http://holoviews.org/reference/streams/bokeh/regression_tap.html

# TODO dynamic regression? https://gist.github.com/darothen/b1538c6571b67ed610669314551d3356

xxx = emfit_df[['avg_hr', 'coverage']].dropna()

# normal.opts(size=10) * hv.Slope.from_scatter(normal)
sc = hv.Scatter((xxx['avg_hr'], xxx['coverage']))
# TODO shit. need to filter nans??
hv.Slope.from_scatter(sc).opts(color='red') * sc.opts(size=10)

In [ ]:
from dashboard.sleep import plot_sleep_correlations
plot_sleep_correlations(emfit_df)

In [ ]:
import holoviews as hv

from dashboard.core.bokeh import _scatter_matrix_demo
sm = _scatter_matrix_demo(width=2000)
show(hv.render(sm))

In [ ]:
# some examples of server with interactive stuff? 
# https://github.com/bokeh/bokeh/blob/master/examples/app/stocks/main.py

# these for sleep intervals? https://docs.bokeh.org/en/latest/docs/gallery/stacked_area.html
# also I had some old jawbone plots I think...
# also existing emfit plots

# food plots (breakdown) https://docs.bokeh.org/en/latest/docs/gallery/bar_stacked.html

# TODO errors should be aggregated and displayed as a separate column? with ability to toggle


# TODO ok, plotly has some callbacks for lasso/rectangle selection https://plotly.com/python/v3/selection-events/
# with these, I could dynamically update the regression lines, I guess


# TODO some nice dynamic interactions https://docs.bokeh.org/en/0.10.0/docs/user_guide/interaction.html

In [ ]:
# todo hmm could reuse tab for it too? although splitting is nicer in terms of caching
import my.body.blood as B
bdf = B.dataframe()

In [ ]:
from dashboard.blood import plot_blood
p = plot_blood(bdf)
show(p)

In [ ]:
from dashboard.data import locations_dataframe
locs = locations_dataframe()

In [ ]:
import pandas as pd
import numpy as np
import geoviews as gv
import geoviews.feature as gf
from cartopy import crs
from bokeh.io import output_notebook
output_notebook()
pd.set_option('display.max_columns', 100)
gv.extension('bokeh')

In [ ]:
from datetime import date
from_ = date(2019, 2, 1)
to    = date(2019, 4, 1)
df = locs[(from_ <= locs.index.date) & (locs.index.date <= to)] # meh, but not too bad

df = df[['lat', 'lon']]
df['date'] = df.index.date # meh
df[:5]
# todo wonder, what's its performance? how many points it can handle?
# ok, 2 month of data took about a minute to render

In [ ]:
# https://malouche.github.io/notebooks/geoviews2.html
# https://data-dive.com/interactive-maps-made-easy-geoviews
plot = (gf.ocean * gf.land * gf.coastline * gf.borders).options(
    'Feature', projection=crs.Mercator(), global_extent=True, frame_height=1000,frame_width=1500,
)
# Convert locations to points and overlay on previous map
points = gv.Points(df, kdims=['lon', 'lat'], vdims=['date']).groupby(['date'])
plot_points = plot * points.opts(alpha=0.3, color='red', size=4)
plot_points.opts(width=500, height=400)
# todo hmm, how to set initial zoom? ideally, auto adjust it interactively depending on the day
# todo how to add proper keybindings for move/zoom? with ctrl etc